In [5]:
import pickle
import numpy as np
import pandas as pd

In [58]:
test1 = pickle.load(open('test1.p','rb'))

In [59]:
docs = pickle.load(open('docs.p','rb'))

In [60]:
models = pickle.load(open('models.p','rb'))

In [62]:
predictions = pickle.load(open('predictions.p', 'rb'))

In [63]:
def docprobmax(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    #***** so HERE is where I need to add the thing.
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").max()
    return prob

def docprobmedian(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    #***** so HERE is where I need to add the thing.
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").median()
    return prob

def docprobmin(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    #***** so HERE is where I need to add the thing.
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").min()
    return prob

#Doesn't work
def docprobmode(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    #***** so HERE is where I need to add the thing.
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = pd.mode(prob.groupby("doc"))
    return prob

In [64]:
probsmedian = docprobmedian(docs, models)
#probsmode = docprobmode(docs[0:2], models)

A numpy array of ones for the number of docs that we're predicting:

In [65]:
predsmedian = np.ones((probsmedian.shape[0]))

In [66]:
print np.shape(predsmedian)

(12500,)


In [67]:
predsmedian[np.where(probsmedian.iloc[:,0] > 0.5)] = 0

In [68]:
predsmedian

array([ 1.,  1.,  0., ...,  1.,  0.,  0.])

In [69]:
print predsmedian.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [70]:
print np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
print np.size(np.where(predsmedian == test1["sentiment"]))*1./np.size(predsmedian)

0.87376
0.85408


Now repeat measuring performance for min and max methods: (Note: min, max, and maxdiff methods are the same.)

In [71]:
probsmax = docprobmax(docs, models)
probsmin = docprobmin(docs, models)

In [72]:
predsmin = np.ones((probsmin.shape[0]))
predsmax = np.ones((probsmax.shape[0]))

In [73]:
#want the largest number for both
predsmin[np.where(probsmin.iloc[:,0] > probsmin.iloc[:,1])] = 0
predsmax[np.where(probsmax.iloc[:,0] > probsmax.iloc[:,1])] = 0

In [74]:
print predsmax.shape
print predsmin.shape
print test1["sentiment"].shape

(12500,)
(12500,)
(12500,)


Note: identical. Mean method still tops.

In [75]:
print np.size(np.where(predsmin == test1["sentiment"]))*1./np.size(predsmin)
print np.size(np.where(predsmax == test1["sentiment"]))*1./np.size(predsmax)

0.85504
0.85512


### MaxDiff/MinDiff method:

In [76]:
#Define function

def docprobmaxdiff(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    #add a diff column
    prob["diff"] = abs(prob[0]-prob[1])
    #now we get the rows with the max diff for each doc
    prob = prob.iloc[prob.groupby(["doc"])["diff"].idxmax()]
    return prob

Testing on subset:

In [81]:
probsmaxdiff = docprobmaxdiff(docs[0:2],models)

In [82]:
print probsmaxdiff

           0         1  doc      diff
1   0.000251  0.999749    0  0.999499
24  0.001323  0.998677    1  0.997354


The real deal:

In [89]:
probsmaxdiff = docprobmaxdiff(docs, models)

In [90]:
predsmaxdiff = np.ones((probsmaxdiff.shape[0]))

In [91]:
#Need to change this line
predsmaxdiff[np.where(probsmaxdiff.iloc[:,0] > probsmaxdiff.iloc[:,1])] = 0

In [92]:
print predsmaxdiff.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [93]:
print np.size(np.where(predsmaxdiff == test1["sentiment"]))*1./np.size(predsmaxdiff)

0.85496


In [94]:
pickle.dump(probsmaxdiff, open('probsmaxdiff.p', 'wb'))
pickle.dump(predsmaxdiff, open('predsmaxdiff.p','wb'))

In [98]:
print predsmaxdiff[0:10]

[ 1.  1.  1.  0.  1.  1.  0.  1.  0.  1.]
